## IMPORT LIBRARIES

In [1]:
import time
start = time.time()

In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict

from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm #SVR is in SVM
from sklearn.metrics import accuracy_score, confusion_matrix

## MAES Feature Influence Test - LENGTH

### Influence test on using Length feature only

In [224]:
dataset = pd.read_csv('maes_features.csv')
# LENGTH :chars, words, commas, apostrophes, punctuations, avg_word_length, sentences, questions, avg_word_sentence 
# PROMPT :prompt_words, prompt_words/total_words, synonym_words, synonym_words/total_words
# BoW: unstemmed, stemmed
# POS (GRAMMATICAL): POS, POS/total_words

In [225]:
dataset

,Unnamed: 0,chars,words,commas,apostrophes,punctuations,avg_word_length,POS,POS/total_words,prompt_words,prompt_words/total_words,synonym_words,synonym_words/total_words,unstemmed,stemmed,score
0,0,2639.0,527.0,15.0,13.0,21.0,5.007590,32.791587,0.062223,220.0,0.417457,112.0,0.212524,584,559,4
1,1,841.0,180.0,5.0,2.0,3.0,4.672222,17.866290,0.099257,82.0,0.455556,66.0,0.366667,210,210,1
2,2,1181.0,261.0,12.0,15.0,14.0,4.524904,22.171206,0.084947,144.0,0.551724,83.0,0.318008,291,285,2
3,3,2705.0,527.0,22.0,6.0,31.0,5.132827,7.026769,0.013334,245.0,0.464896,131.0,0.248577,547,528,4
4,4,2394.0,501.0,25.0,15.0,34.0,4.778443,31.795655,0.063464,216.0,0.431138,117.0,0.233533,591,562,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,1795,1339.0,286.0,0.0,34.0,11.0,4.681818,31.891253,0.111508,114.0,0.398601,78.0,0.272727,303,293,3
1796,1796,1373.0,282.0,15.0,8.0,15.0,4.868794,12.757794,0.045240,152.0,0.539007,80.0,0.283688,339,333,3
1797,1797,605.0,132.0,7.0,2.0,5.0,4.583333,3.387097,0.025660,71.0,0.537879,34.0,0.257576,164,160,2
1798,1798,2737.0,561.0,22.0,5.0,27.0,4.878788,20.071942,0.035779,301.0,0.536542,168.0,0.299465,542,512,3


take only `length` features.

Including chars, words, commas, apostrophes, punctuations, avg_word_length, sentences, questions, avg_word_sentence 

In [226]:
X = dataset.iloc[:,1:7].values.astype(float)
y = dataset.iloc[:,15].values.astype(float)
y

array([4., 1., 2., ..., 2., 3., 3.])

In [228]:
y.shape

(1800,)

In [229]:
y = np.array(y).reshape(-1,1)
y.shape

(1800, 1)

In [230]:
### Split the train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# Have a look at the first few lines
print(y_test[:5, :])

[[3.]
 [5.]
 [3.]
 [4.]
 [4.]]


## Model Training using `Length` features only

### Naive Bayes Training 

In [174]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [175]:
X_trainNB = X_train
y_trainNB = y_train
X_testNB = X_test
y_testNB = y_test

In [176]:
model_nbl = naive_bayes.MultinomialNB()
model_nbl.fit(X_trainNB, y_trainNB.ravel())

MultinomialNB()

The Naive Bayes model using `length` features only is called `model_nbl`

### SVM Training using `Length` features only

In [177]:
from sklearn.preprocessing import StandardScaler
sc_Xsvm = StandardScaler()
sc_ysvm = StandardScaler()
X_trainSVM = sc_Xsvm.fit_transform(X_train)
y_trainSVM = sc_ysvm.fit_transform(y_train)
X_testSVM = sc_Xsvm.transform(X_test)
y_testSVM = sc_ysvm.transform(y_test)

In [178]:
from sklearn.svm import SVR
# most important SVR parameter is Kernel type. It can be #linear,polynomial or gaussian SVR. We have a non-linear condition #so we can select polynomial or gaussian but here we select RBF(a #gaussian type) kernel.
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
# maybe use poly and increase the degree
model_svml = SVR(kernel='rbf', gamma='auto', verbose=True)
#regressor = SVR(kernel='poly', degree=5, gamma='auto', verbose=True)
model_svml.fit(X_trainSVM,y_trainSVM.ravel())

[LibSVM]

SVR(gamma='auto', verbose=True)

The Support Vector Machine (SVM) model using `length` features only is now called `model_svml`

### BLRR


In [179]:
from sklearn.preprocessing import StandardScaler
sc_Xblrr = StandardScaler()
sc_yblrr = StandardScaler()
X_trainBLRR = sc_Xblrr.fit_transform(X_train)
y_trainBLRR = sc_yblrr.fit_transform(y_train)
X_testBLRR = sc_Xblrr.transform(X_test)
y_testBLRR = sc_yblrr.transform(y_test)

In [180]:
from sklearn import linear_model
model_blrrl = linear_model.BayesianRidge()
model_blrrl.fit(X_trainBLRR, y_trainBLRR.ravel())

BayesianRidge()

At this stage, the Bayesian Linear Ridge Regression (BLRR) model using `length` feature only is called `model_blrrl`


## Prediction using `length` feature only

### Naive Bayes `length` only

In [181]:
y_predNBL = model_nbl.predict(X_testNB)
#y_1_predNB = sc_ynb.inverse_transform(y_1_predNB).round()
#cm = confusion_matrix(y_test, y_predNBL)
#print(cm)

### SVM `length` only

In [182]:
y_predSVML = model_svml.predict(X_testSVM)
y_predSVML = sc_ysvm.inverse_transform(y_predSVML).round()

#cm = confusion_matrix(y_test, y_predSVML)
#np.set_printoptions(threshold=np.inf)
#print(cm)

### BLRR `length` only

In [183]:
y_predBLRRL = model_blrrl.predict(X_testBLRR)
y_predBLRRL = sc_yblrr.inverse_transform(y_predBLRRL).round()

#cm = confusion_matrix(y_test, y_predBLRRL)
#print(cm)

## Ensembling the 3 Algo using `length` features only

In [184]:
actual = pd.Series(y_test.ravel())
predNBL = pd.Series(y_predNBL)
predSVML = pd.Series(y_predSVML)
predBLRRL = pd.Series(y_predBLRRL)

data = {"Actual": actual,
        "NBL": predNBL, 
        "SVML": predSVML, 
        "BLRRL": predBLRRL} 
results = pd.concat(data, axis=1)
results.head(5)

,Actual,NBL,SVML,BLRRL
0,3.0,3.0,4.0,5.0
1,5.0,4.0,4.0,4.0
2,3.0,3.0,3.0,3.0
3,4.0,3.0,3.0,4.0
4,4.0,4.0,4.0,4.0


In [185]:
results['EnsembleL'] = np.where(
                            (results['NBL'] == results['BLRRL']) |
                            (results['NBL'] == results['SVML']),
                            results['NBL'],
                            results['BLRRL']
                        )
results.head(5)

,Actual,NBL,SVML,BLRRL,EnsembleL
0,3.0,3.0,4.0,5.0,5.0
1,5.0,4.0,4.0,4.0,4.0
2,3.0,3.0,3.0,3.0,3.0
3,4.0,3.0,3.0,4.0,3.0
4,4.0,4.0,4.0,4.0,4.0


## Evaluation using QWK

In [186]:
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score

### Naive Bayes

In [187]:
rpt = classification_report(y_test, y_predNBL)
print(rpt)

              precision    recall  f1-score   support

         1.0       0.50      0.14      0.22         7
         2.0       0.23      0.22      0.22        23
         3.0       0.59      0.65      0.62       160
         4.0       0.62      0.60      0.61       156
         5.0       0.50      0.31      0.38        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.58       360
   macro avg       0.41      0.32      0.34       360
weighted avg       0.57      0.58      0.57       360



C:\Users\User\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [188]:
print(cohen_kappa_score(y_test, y_predNBL, weights="quadratic"))

0.504200146092038


### SVM 

In [189]:
rpt = classification_report(y_test, y_predSVML)
print(rpt)

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.50      0.35      0.41        23
         3.0       0.63      0.74      0.68       160
         4.0       0.67      0.65      0.66       156
         5.0       0.67      0.15      0.25        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.64       360
   macro avg       0.41      0.32      0.33       360
weighted avg       0.62      0.64      0.62       360



In [190]:
print(cohen_kappa_score(y_test, y_predSVML, weights="quadratic"))

0.5801449859041481


### BLRR

In [191]:
rpt = classification_report(y_test, y_predBLRRL)
print(rpt)

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.50      0.30      0.38        23
         3.0       0.62      0.81      0.70       160
         4.0       0.74      0.60      0.66       156
         5.0       0.58      0.54      0.56        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.66       360
   macro avg       0.41      0.37      0.38       360
weighted avg       0.65      0.66      0.64       360



In [192]:
print(cohen_kappa_score(y_test, y_predBLRRL, weights="quadratic"))

0.6036624002027626


### Ensemble

In [30]:
rpt = classification_report(y_test,results['EnsembleL'])
print(rpt)

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.47      0.30      0.37        23
         3.0       0.63      0.78      0.69       160
         4.0       0.70      0.63      0.66       156
         5.0       0.60      0.23      0.33        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.65       360
   macro avg       0.40      0.32      0.34       360
weighted avg       0.63      0.65      0.63       360



In [31]:
print(cohen_kappa_score(y_test, results['EnsembleL'], weights="quadratic"))

0.598688524590164


#### QWK Scores result with `length` features only:

Naive Bayes: 0.504200146092038

SVM: 0.5801449859041481

BLRR: 0.6036624002027626

Ensemble: 0.598688524590164


## `POS` influence test 

In [92]:
dataset = pd.read_csv('maes_features.csv')
# LENGTH :chars, words, commas, apostrophes, punctuations, avg_word_length, sentences, questions, avg_word_sentence 
# PROMPT :prompt_words, prompt_words/total_words, synonym_words, synonym_words/total_words
# BoW: unstemmed, stemmed
# POS (GRAMMATICAL): POS, POS/total_words

In [93]:
dataset

,Unnamed: 0,chars,words,commas,apostrophes,punctuations,avg_word_length,POS,POS/total_words,prompt_words,prompt_words/total_words,synonym_words,synonym_words/total_words,unstemmed,stemmed,score
0,0,2639.0,527.0,15.0,13.0,21.0,5.007590,524.330784,0.994935,220.0,0.417457,112.0,0.212524,584,559,4
1,1,841.0,180.0,5.0,2.0,3.0,4.672222,178.662900,0.992572,82.0,0.455556,66.0,0.366667,210,210,1
2,2,1181.0,261.0,12.0,15.0,14.0,4.524904,257.992218,0.988476,144.0,0.551724,83.0,0.318008,291,285,2
3,3,2705.0,527.0,22.0,6.0,31.0,5.132827,521.653920,0.989856,245.0,0.464896,131.0,0.248577,547,528,4
4,4,2394.0,501.0,25.0,15.0,34.0,4.778443,484.298031,0.966663,216.0,0.431138,117.0,0.233533,591,562,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,1795,1339.0,286.0,0.0,34.0,11.0,4.681818,280.643026,0.981269,114.0,0.398601,78.0,0.272727,303,293,3
1796,1796,1373.0,282.0,15.0,8.0,15.0,4.868794,277.649880,0.984574,152.0,0.539007,80.0,0.283688,339,333,3
1797,1797,605.0,132.0,7.0,2.0,5.0,4.583333,125.322581,0.949413,71.0,0.537879,34.0,0.257576,164,160,2
1798,1798,2737.0,561.0,22.0,5.0,27.0,4.878788,551.978417,0.983919,301.0,0.536542,168.0,0.299465,542,512,3


take only `PoS` features.

Including PoS, Pos/total_words

In [94]:
X = dataset.iloc[:,7:9].values.astype(float)
y = dataset.iloc[:,15].values.astype(float)
X

array([[524.33078394,   0.99493507],
       [178.66290019,   0.99257167],
       [257.9922179 ,   0.98847593],
       ...,
       [125.32258065,   0.94941349],
       [551.97841727,   0.98391875],
       [511.32291667,   0.9833133 ]])

In [95]:
X.shape

(1800, 2)

In [96]:
y = np.array(y).reshape(-1,1)
y.shape

(1800, 1)

In [97]:
### Split the train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# Have a look at the first few lines
print(y_test[:5, :])

[[3.]
 [5.]
 [3.]
 [4.]
 [4.]]


### Model Training using `PoS` features only

### Naive Bayes Training

In [98]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

X_trainNB = X_train
y_trainNB = y_train
X_testNB = X_test
y_testNB = y_test

model_nb = naive_bayes.MultinomialNB()
model_nb.fit(X_trainNB, y_trainNB.ravel())

MultinomialNB()

### SVM Training

In [99]:
from sklearn.preprocessing import StandardScaler
sc_Xsvm = StandardScaler()
sc_ysvm = StandardScaler()
X_trainSVM = sc_Xsvm.fit_transform(X_train)
y_trainSVM = sc_ysvm.fit_transform(y_train)
X_testSVM = sc_Xsvm.transform(X_test)
y_testSVM = sc_ysvm.transform(y_test)


from sklearn.svm import SVR
# most important SVR parameter is Kernel type. It can be #linear,polynomial or gaussian SVR. We have a non-linear condition #so we can select polynomial or gaussian but here we select RBF(a #gaussian type) kernel.
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
# maybe use poly and increase the degree
model_svm = SVR(kernel='rbf', gamma='auto', verbose=True)
#regressor = SVR(kernel='poly', degree=5, gamma='auto', verbose=True)
model_svm.fit(X_trainSVM,y_trainSVM.ravel())

[LibSVM]

SVR(gamma='auto', verbose=True)

### BLRR Training

In [100]:
from sklearn.preprocessing import StandardScaler
sc_Xblrr = StandardScaler()
sc_yblrr = StandardScaler()
X_trainBLRR = sc_Xblrr.fit_transform(X_train)
y_trainBLRR = sc_yblrr.fit_transform(y_train)
X_testBLRR = sc_Xblrr.transform(X_test)
y_testBLRR = sc_yblrr.transform(y_test)

from sklearn import linear_model
model_blrr = linear_model.BayesianRidge()
model_blrr.fit(X_trainBLRR, y_trainBLRR.ravel())

BayesianRidge()

## Prediction using `pos` only

In [101]:
#NB
y_predNB = model_nb.predict(X_testNB)
#y_1_predNB = sc_ynb.inverse_transform(y_1_predNB).round()

cm = confusion_matrix(y_test, y_predNB)

print(cm)

[[  0   0   7   0   0   0]
 [  0   0  22   1   0   0]
 [  0   0 108  52   0   0]
 [  0   0  33 123   0   0]
 [  0   0   0  13   0   0]
 [  0   0   0   1   0   0]]


In [102]:
#SVM
y_predSVM = model_svm.predict(X_testSVM)
y_predSVM = sc_ysvm.inverse_transform(y_predSVM).round()

#cm = confusion_matrix(y_test, y_predSVML)
#np.set_printoptions(threshold=np.inf)
#print(cm)

In [103]:
#BLRR
y_predBLRR = model_blrr.predict(X_testBLRR)
y_predBLRR = sc_yblrr.inverse_transform(y_predBLRR).round()

#cm = confusion_matrix(y_test, y_predBLRRL)
#print(cm)

In [104]:
actual = pd.Series(y_test.ravel())
predNB = pd.Series(y_predNB)
predSVM = pd.Series(y_predSVM)
predBLRR = pd.Series(y_predBLRR)

data = {"Actual": actual,
        "NB": predNB, 
        "SVM": predSVM, 
        "BLRR": predBLRR} 
results = pd.concat(data, axis=1)
#results.head(5)

In [105]:
results['Ensemble'] = np.where(
                            (results['NB'] == results['BLRR']) |
                            (results['NB'] == results['SVM']),
                            results['NB'],
                            results['BLRR']
                        )
results.head(5)

,Actual,NB,SVM,BLRR,Ensemble
0,3.0,4.0,4.0,5.0,4.0
1,5.0,4.0,4.0,4.0,4.0
2,3.0,3.0,3.0,3.0,3.0
3,4.0,4.0,3.0,3.0,3.0
4,4.0,4.0,4.0,4.0,4.0


## Evaluation `pos` only

In [106]:
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score


### NB

In [107]:
rpt = classification_report(y_test, y_predNB)
print(rpt)

print(cohen_kappa_score(y_test, y_predNB, weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.00      0.00      0.00        23
         3.0       0.64      0.68      0.65       160
         4.0       0.65      0.79      0.71       156
         5.0       0.00      0.00      0.00        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.64       360
   macro avg       0.21      0.24      0.23       360
weighted avg       0.56      0.64      0.60       360

0.4830633284241532


C:\Users\User\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SVM

In [108]:
rpt = classification_report(y_test, y_predSVM)
print(rpt)

print(cohen_kappa_score(y_test, y_predSVM, weights="quadratic"))

              precision    recall  f1-score   support

         1.0       1.00      0.14      0.25         7
         2.0       0.54      0.30      0.39        23
         3.0       0.64      0.78      0.71       160
         4.0       0.68      0.67      0.68       156
         5.0       0.00      0.00      0.00        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.66       360
   macro avg       0.48      0.32      0.34       360
weighted avg       0.64      0.66      0.64       360

0.5837900794738284


### BLRR

In [109]:
rpt = classification_report(y_test, y_predBLRR)
print(rpt)

print(cohen_kappa_score(y_test, y_predBLRR, weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.33      0.13      0.19        23
         3.0       0.60      0.81      0.69       160
         4.0       0.70      0.56      0.62       156
         5.0       0.50      0.38      0.43        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.62       360
   macro avg       0.36      0.31      0.32       360
weighted avg       0.61      0.62      0.60       360

0.5363636363636364


### Ensemble

In [110]:
rpt = classification_report(y_test,results['Ensemble'])
print(rpt)

print(cohen_kappa_score(y_test, results['Ensemble'], weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.33      0.13      0.19        23
         3.0       0.64      0.79      0.70       160
         4.0       0.69      0.67      0.68       156
         5.0       0.00      0.00      0.00        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.65       360
   macro avg       0.28      0.27      0.26       360
weighted avg       0.60      0.65      0.62       360

0.5571245186136072


#### QWK Scores result with `PoS` features only:

Naive Bayes: 0.4830633284241532

SVM: 0.5837900794738284

BLRR: 0.5363636363636364

Ensemble: 0.5571245186136072


## `BoW` influence test 

In [207]:
dataset = pd.read_csv('maes_features.csv')
# LENGTH :chars, words, commas, apostrophes, punctuations, avg_word_length, sentences, questions, avg_word_sentence 
# PROMPT :prompt_words, prompt_words/total_words, synonym_words, synonym_words/total_words
# BoW: unstemmed, stemmed
# POS (GRAMMATICAL): POS, POS/total_words

In [208]:
dataset

,Unnamed: 0,chars,words,commas,apostrophes,punctuations,avg_word_length,POS,POS/total_words,prompt_words,prompt_words/total_words,synonym_words,synonym_words/total_words,unstemmed,stemmed,score
0,0,2639.0,527.0,15.0,13.0,21.0,5.007590,32.791587,0.062223,220.0,0.417457,112.0,0.212524,584,559,4
1,1,841.0,180.0,5.0,2.0,3.0,4.672222,17.866290,0.099257,82.0,0.455556,66.0,0.366667,210,210,1
2,2,1181.0,261.0,12.0,15.0,14.0,4.524904,22.171206,0.084947,144.0,0.551724,83.0,0.318008,291,285,2
3,3,2705.0,527.0,22.0,6.0,31.0,5.132827,7.026769,0.013334,245.0,0.464896,131.0,0.248577,547,528,4
4,4,2394.0,501.0,25.0,15.0,34.0,4.778443,31.795655,0.063464,216.0,0.431138,117.0,0.233533,591,562,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,1795,1339.0,286.0,0.0,34.0,11.0,4.681818,31.891253,0.111508,114.0,0.398601,78.0,0.272727,303,293,3
1796,1796,1373.0,282.0,15.0,8.0,15.0,4.868794,12.757794,0.045240,152.0,0.539007,80.0,0.283688,339,333,3
1797,1797,605.0,132.0,7.0,2.0,5.0,4.583333,3.387097,0.025660,71.0,0.537879,34.0,0.257576,164,160,2
1798,1798,2737.0,561.0,22.0,5.0,27.0,4.878788,20.071942,0.035779,301.0,0.536542,168.0,0.299465,542,512,3


take only `BoW` features.

Including unstemmed, stemmed

In [209]:
X = dataset.iloc[:,13:15].values.astype(float)
y = dataset.iloc[:,15].values.astype(float)
X

array([[584., 559.],
       [210., 210.],
       [291., 285.],
       ...,
       [164., 160.],
       [542., 512.],
       [514., 496.]])

In [210]:
X.shape

y = np.array(y).reshape(-1,1)
y.shape

### Split the train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# Have a look at the first few lines
print(X_train[:5, :])

[[310. 296.]
 [348. 339.]
 [ 83.  83.]
 [647. 625.]
 [593. 564.]]


### Model Training using `BoW` features only

### NB

In [211]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

X_trainNB = X_train
y_trainNB = y_train
X_testNB = X_test
y_testNB = y_test

model_nb = naive_bayes.MultinomialNB()
model_nb.fit(X_trainNB, y_trainNB.ravel())

MultinomialNB()

### SVM

In [212]:
from sklearn.preprocessing import StandardScaler
sc_Xsvm = StandardScaler()
sc_ysvm = StandardScaler()
X_trainSVM = sc_Xsvm.fit_transform(X_train)
y_trainSVM = sc_ysvm.fit_transform(y_train)
X_testSVM = sc_Xsvm.transform(X_test)
y_testSVM = sc_ysvm.transform(y_test)


from sklearn.svm import SVR
# most important SVR parameter is Kernel type. It can be #linear,polynomial or gaussian SVR. We have a non-linear condition #so we can select polynomial or gaussian but here we select RBF(a #gaussian type) kernel.
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
# maybe use poly and increase the degree
model_svm = SVR(kernel='rbf', gamma='auto', verbose=True)
#regressor = SVR(kernel='poly', degree=5, gamma='auto', verbose=True)
model_svm.fit(X_trainSVM,y_trainSVM.ravel())

[LibSVM]

SVR(gamma='auto', verbose=True)

### BLRR

In [213]:
from sklearn.preprocessing import StandardScaler
sc_Xblrr = StandardScaler()
sc_yblrr = StandardScaler()
X_trainBLRR = sc_Xblrr.fit_transform(X_train)
y_trainBLRR = sc_yblrr.fit_transform(y_train)
X_testBLRR = sc_Xblrr.transform(X_test)
y_testBLRR = sc_yblrr.transform(y_test)

from sklearn import linear_model
model_blrr = linear_model.BayesianRidge()
model_blrr.fit(X_trainBLRR, y_trainBLRR.ravel())

BayesianRidge()

## Prediction using `BoW` only

In [214]:
#NB
y_predNB = model_nb.predict(X_testNB)
#y_1_predNB = sc_ynb.inverse_transform(y_1_predNB).round()

cm = confusion_matrix(y_testNB, y_predNB)

print(cm)

[[  0   0   0   7   0   0]
 [  0   0   0  23   0   0]
 [  0   0   0 160   0   0]
 [  0   0   0 156   0   0]
 [  0   0   0  13   0   0]
 [  0   0   0   1   0   0]]


In [215]:
#SVM
y_predSVM = model_svm.predict(X_testSVM)
y_predSVM = sc_ysvm.inverse_transform(y_predSVM).round()

cm = confusion_matrix(y_test, y_predSVM)
#np.set_printoptions(threshold=np.inf)
print(cm)

[[  0   6   1   0   0   0]
 [  0  10  12   1   0   0]
 [  0   3 115  42   0   0]
 [  0   0  48 108   0   0]
 [  0   0   0  13   0   0]
 [  0   0   0   1   0   0]]


In [216]:
#BLRR
y_predBLRR = model_blrr.predict(X_testBLRR)
y_predBLRR = sc_yblrr.inverse_transform(y_predBLRR).round()

cm = confusion_matrix(y_test, y_predBLRRL)
print(cm)

[[  0   5   2   0   0   0]
 [  0   7  15   1   0   0]
 [  0   2 129  27   2   0]
 [  0   0  61  93   2   0]
 [  0   0   0   5   7   1]
 [  0   0   0   0   1   0]]


In [217]:
actual = pd.Series(y_test.ravel())
predNB = pd.Series(y_predNB)
predSVM = pd.Series(y_predSVM)
predBLRR = pd.Series(y_predBLRR)

data = {"Actual": actual,
        "NB": predNB, 
        "SVM": predSVM, 
        "BLRR": predBLRR} 
results = pd.concat(data, axis=1)
results.head(5)

,Actual,NB,SVM,BLRR
0,3.0,4.0,4.0,4.0
1,5.0,4.0,4.0,4.0
2,3.0,4.0,3.0,3.0
3,4.0,4.0,4.0,4.0
4,4.0,4.0,4.0,4.0


In [218]:
results['Ensemble'] = np.where(
                            (results['NB'] == results['BLRR']) |
                            (results['NB'] == results['SVM']),
                            results['NB'],
                            results['BLRR']
                        )
results.head(5)

,Actual,NB,SVM,BLRR,Ensemble
0,3.0,4.0,4.0,4.0,4.0
1,5.0,4.0,4.0,4.0,4.0
2,3.0,4.0,3.0,3.0,3.0
3,4.0,4.0,4.0,4.0,4.0
4,4.0,4.0,4.0,4.0,4.0


## Evaluation `BoW` only

In [219]:
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score


### NB

In [220]:
rpt = classification_report(y_test, y_predNB)
print(rpt)

print(cohen_kappa_score(y_test, y_predNB, weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.00      0.00      0.00        23
         3.0       0.00      0.00      0.00       160
         4.0       0.43      1.00      0.60       156
         5.0       0.00      0.00      0.00        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.43       360
   macro avg       0.07      0.17      0.10       360
weighted avg       0.19      0.43      0.26       360

0.0


C:\Users\User\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SVM

In [221]:
rpt = classification_report(y_test, y_predSVM)
print(rpt)

print(cohen_kappa_score(y_test, y_predSVM, weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.53      0.43      0.48        23
         3.0       0.65      0.72      0.68       160
         4.0       0.65      0.69      0.67       156
         5.0       0.00      0.00      0.00        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.65       360
   macro avg       0.31      0.31      0.31       360
weighted avg       0.61      0.65      0.63       360

0.5903066006158789


### BLRR

In [222]:
rpt = classification_report(y_test, y_predBLRR)
print(rpt)

print(cohen_kappa_score(y_test, y_predBLRR, weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.57      0.35      0.43        23
         3.0       0.63      0.79      0.70       160
         4.0       0.68      0.63      0.65       156
         5.0       0.00      0.00      0.00        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.65       360
   macro avg       0.31      0.29      0.30       360
weighted avg       0.61      0.65      0.62       360

0.5738161559888579


### Ensemble

In [223]:
rpt = classification_report(y_test,results['Ensemble'])
print(rpt)

print(cohen_kappa_score(y_test, results['Ensemble'], weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.57      0.35      0.43        23
         3.0       0.65      0.73      0.69       160
         4.0       0.65      0.69      0.67       156
         5.0       0.00      0.00      0.00        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.65       360
   macro avg       0.31      0.30      0.30       360
weighted avg       0.61      0.65      0.62       360

0.5694520925110131



#### QWK Scores result with `BoW` features only:

Naive Bayes: 0.0

SVM: 0.5903066006158789

BLRR: 0.5738161559888579

Ensemble: 0.5694520925110131

#### Conclusion

With `BoW` only, it significantly dropping Naive Bayes model's QWK score to 0. The SVM and BLRR also worsen but in a smaller margin.

On the output of the QWK agreements, the Naive bayes (BoW only) QWK score is 'agreement equivalent to chance' which makes the model not working at all.

## `Prompt` influence test 

In [152]:
dataset = pd.read_csv('maes_features.csv')
# LENGTH :chars, words, commas, apostrophes, punctuations, avg_word_length, sentences, questions, avg_word_sentence 
# PROMPT :prompt_words, prompt_words/total_words, synonym_words, synonym_words/total_words
# BoW: unstemmed, stemmed
# POS (GRAMMATICAL): POS, POS/total_words

take only `Prompt` features.

Including prompt_words, prompt_words/total_words, synonym_words, synonym_words/total_words

In [153]:
X = dataset.iloc[:,9:13].values.astype(float)
y = dataset.iloc[:,15].values.astype(float)
X

array([[2.20000000e+02, 4.17457306e-01, 1.12000000e+02, 2.12523719e-01],
       [8.20000000e+01, 4.55555556e-01, 6.60000000e+01, 3.66666667e-01],
       [1.44000000e+02, 5.51724138e-01, 8.30000000e+01, 3.18007663e-01],
       ...,
       [7.10000000e+01, 5.37878788e-01, 3.40000000e+01, 2.57575758e-01],
       [3.01000000e+02, 5.36541889e-01, 1.68000000e+02, 2.99465241e-01],
       [2.31000000e+02, 4.44230769e-01, 1.65000000e+02, 3.17307692e-01]])

In [154]:
X.shape

y = np.array(y).reshape(-1,1)
y.shape

### Split the train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# Have a look at the first few lines
print(y_test[:5, :])

[[3.]
 [5.]
 [3.]
 [4.]
 [4.]]


### Model Training using `Prompt` features only

In [155]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

X_trainNB = X_train
y_trainNB = y_train
X_testNB = X_test
y_testNB = y_test

model_nb = naive_bayes.MultinomialNB()
model_nb.fit(X_trainNB, y_trainNB.ravel())

MultinomialNB()

In [156]:
from sklearn.preprocessing import StandardScaler
sc_Xsvm = StandardScaler()
sc_ysvm = StandardScaler()
X_trainSVM = sc_Xsvm.fit_transform(X_train)
y_trainSVM = sc_ysvm.fit_transform(y_train)
X_testSVM = sc_Xsvm.transform(X_test)
y_testSVM = sc_ysvm.transform(y_test)


from sklearn.svm import SVR
# most important SVR parameter is Kernel type. It can be #linear,polynomial or gaussian SVR. We have a non-linear condition #so we can select polynomial or gaussian but here we select RBF(a #gaussian type) kernel.
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
# maybe use poly and increase the degree
model_svm = SVR(kernel='rbf', gamma='auto', verbose=True)
#regressor = SVR(kernel='poly', degree=5, gamma='auto', verbose=True)
model_svm.fit(X_trainSVM,y_trainSVM.ravel())

[LibSVM]

SVR(gamma='auto', verbose=True)

In [157]:
from sklearn.preprocessing import StandardScaler
sc_Xblrr = StandardScaler()
sc_yblrr = StandardScaler()
X_trainBLRR = sc_Xblrr.fit_transform(X_train)
y_trainBLRR = sc_yblrr.fit_transform(y_train)
X_testBLRR = sc_Xblrr.transform(X_test)
y_testBLRR = sc_yblrr.transform(y_test)

from sklearn import linear_model
model_blrr = linear_model.BayesianRidge()
model_blrr.fit(X_trainBLRR, y_trainBLRR.ravel())

BayesianRidge()

## Predict `Prompt` only

In [158]:
#NB
y_predNB = model_nb.predict(X_testNB)
#y_1_predNB = sc_ynb.inverse_transform(y_1_predNB).round()

cm = confusion_matrix(y_test, y_predNB)

print(cm)

[[ 0  0  7  0  0  0]
 [ 0  0 15  8  0  0]
 [ 0  0 90 70  0  0]
 [ 0  0 62 94  0  0]
 [ 0  0  1 12  0  0]
 [ 0  0  0  1  0  0]]


In [159]:
#SVM
y_predSVM = model_svm.predict(X_testSVM)
y_predSVM = sc_ysvm.inverse_transform(y_predSVM).round()

cm = confusion_matrix(y_test, y_predSVM)
#np.set_printoptions(threshold=np.inf)
print(cm)

[[  0   5   2   0   0   0]
 [  0   9  13   1   0   0]
 [  0   2 120  38   0   0]
 [  0   0  53 103   0   0]
 [  0   0   0  13   0   0]
 [  0   0   0   1   0   0]]


In [160]:
#BLRR
y_predBLRR = model_blrr.predict(X_testBLRR)
y_predBLRR = sc_yblrr.inverse_transform(y_predBLRR).round()

cm = confusion_matrix(y_test, y_predBLRR)
print(cm)

[[  0   3   4   0   0   0]
 [  0   9  13   1   0   0]
 [  0   3 127  28   2   0]
 [  0   0  64  90   2   0]
 [  0   0   0   9   4   0]
 [  0   0   0   1   0   0]]


In [161]:
actual = pd.Series(y_test.ravel())
predNB = pd.Series(y_predNB)
predSVM = pd.Series(y_predSVM)
predBLRR = pd.Series(y_predBLRR)

data = {"Actual": actual,
        "NB": predNB, 
        "SVM": predSVM, 
        "BLRR": predBLRR} 
results = pd.concat(data, axis=1)
results.head(5)

,Actual,NB,SVM,BLRR
0,3.0,3.0,4.0,5.0
1,5.0,4.0,4.0,4.0
2,3.0,3.0,3.0,3.0
3,4.0,3.0,3.0,3.0
4,4.0,4.0,4.0,4.0


In [162]:
results['Ensemble'] = np.where(
                            (results['NB'] == results['BLRR']) |
                            (results['NB'] == results['SVM']),
                            results['NB'],
                            results['BLRR']
                        )
results.head(5)

,Actual,NB,SVM,BLRR,Ensemble
0,3.0,3.0,4.0,5.0,5.0
1,5.0,4.0,4.0,4.0,4.0
2,3.0,3.0,3.0,3.0,3.0
3,4.0,3.0,3.0,3.0,3.0
4,4.0,4.0,4.0,4.0,4.0


## Evaluation `Prompt` only

In [163]:
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score


## NB

In [164]:
rpt = classification_report(y_test, y_predNB)
print(rpt)

print(cohen_kappa_score(y_test, y_predNB, weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.00      0.00      0.00        23
         3.0       0.51      0.56      0.54       160
         4.0       0.51      0.60      0.55       156
         5.0       0.00      0.00      0.00        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.51       360
   macro avg       0.17      0.19      0.18       360
weighted avg       0.45      0.51      0.48       360

0.24556868537666177


C:\Users\User\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SVM

In [165]:
rpt = classification_report(y_test, y_predSVM)
print(rpt)

print(cohen_kappa_score(y_test, y_predSVM, weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.56      0.39      0.46        23
         3.0       0.64      0.75      0.69       160
         4.0       0.66      0.66      0.66       156
         5.0       0.00      0.00      0.00        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.64       360
   macro avg       0.31      0.30      0.30       360
weighted avg       0.61      0.64      0.62       360

0.5690834473324213


### BLRR

In [166]:
rpt = classification_report(y_test, y_predBLRR)
print(rpt)

print(cohen_kappa_score(y_test, y_predBLRR, weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.60      0.39      0.47        23
         3.0       0.61      0.79      0.69       160
         4.0       0.70      0.58      0.63       156
         5.0       0.50      0.31      0.38        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.64       360
   macro avg       0.40      0.34      0.36       360
weighted avg       0.63      0.64      0.62       360

0.5431773236651285


### Ensemble

In [167]:
rpt = classification_report(y_test,results['Ensemble'])
print(rpt)

print(cohen_kappa_score(y_test, results['Ensemble'], weights="quadratic"))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.62      0.35      0.44        23
         3.0       0.64      0.76      0.69       160
         4.0       0.67      0.66      0.67       156
         5.0       0.00      0.00      0.00        13
         6.0       0.00      0.00      0.00         1

    accuracy                           0.65       360
   macro avg       0.32      0.30      0.30       360
weighted avg       0.61      0.65      0.63       360

0.5426452410383189


#### QWK Scores result with full features:

Naive Bayes: 0.5172740179839092

SVM: 0.6011377166291838

BLRR: 0.6261591962905719

Ensemble: 0.6060822202049021

#### QWK Scores result with `Prompt` features only:

Naive Bayes: 0.24556868537666177

SVM: 0.5690834473324213

BLRR: 0.5431773236651285

Ensemble: 0.5426452410383189


### Final Conclusion - training model based on single type of feature
Below are a table conclude the result of model trained by single type of feature.


| Only Feature Used | Naive Bayes| SVM | BLRR | Ensemble | 
| --- | --- | --- | --- | --- | 
| Length | 0.504 | 0.580 | 0.604 | 0.599 | 
| PoS | 0.483 | 0.584 | 0.536 | 0.557 | 
| BoW | 0.0 | 0.590 | 0.574 | 0.569 | 
| Prompt | 0.246 | 0.569 | 0.543 | 0.542 | 
| All features | 0.517 | 0.601 | 0.626 | 0.606 |

Differences between all features and only features

| Only Features | Naive Bayes| SVM | BLRR | Ensemble | Total Diff (absolute value)  |
| --- | --- | --- | --- | --- | --- |
| Length | 0.013 | 0.021 | 0.022 | 0.007 | 0.063 |
| PoS | 0.034 | 0.017 | 0.090 | 0.049 | 0.190 |
| BoW | 0.517 | 0.011 | 0.052 | 0.037 | 0.617 |
| Prompt | 0.271 | 0.032 | 0.083 | 0.064 | 0.450 |

Higher in differences should mean lower in influence on the model.


`BoW` has no influence at all on Naive Bayes Model.

`Prompt` has the least influence on SVM and second least influence in BLRR.

`Length` has the most influences among all model.